In [1]:
# Directories
import os

new_directory = r'c://Users//Fer//TESIS_ARCHIVOS//TESIS_AIRE//MP_Forecasting//aqi_forecasting//notebooks'
os.chdir(new_directory)

# Data Manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation


# Training utils
from training_code.utils import utils_xgboost

# Optuna
import optuna
import joblib
import pickle

# Tiempo
import datetime as dt
from dateutil.relativedelta import relativedelta, MO

# Modelos
from sklearn.linear_model import LinearRegression # for building a linear regression model
from sklearn.svm import SVR # for building SVR model
from sklearn.preprocessing import MinMaxScaler
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Metricas
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import mean_absolute_percentage_error #MAPE
from sklearn.metrics import mean_squared_error #MSE, para RMSE: squared = False

# Visualizations
import plotly.graph_objects as go # for data visualization
import plotly.express as px # for data visualization
import matplotlib.pyplot as plt

# Advertencias
import warnings
warnings.filterwarnings("ignore")

c:\Users\Fer\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
datos = pd.read_csv('datos/230127_train_ESTACIONES.csv', parse_dates=['FECHAHORA'])

# VENTANAS DE TIEMPO

In [ ]:
horizon = [0.25, 0.5, 1]
resample = [5, 30, 60]
previous = [2, 7, 15]

# Start resampling

for resample_item in resample:

    df_train = datos.copy()

    df_train.drop(['ANHO', 'DIA', 'MES', 'HORA', 'MINUTO', 'DIA_TRAF_COD', 'TRAFICO_COD', 'TIPO_COD',
                'MEDICION_DIA', 'MP1_ANTERIOR', 'MP2_5_ANTERIOR', 'MP10_ANTERIOR',
                'TEMPERATURA_PRONOSTICO', 'HUMEDAD_PRONOSTICO', 'PRESION_PRONOSTICO'], axis = 1, inplace = True)

    df_train = df_train.set_index('FECHAHORA', drop = True)

    lista_resample = []

    if resample_item != 5: 

        r_i = str(resample_item) + 'T'

        for station in range(1,11):
            df_aux = df_train[df_train['ESTACION'] == station]

            df_aux = df_aux.resample(r_i).mean()
            lista_resample.append(df_aux)

        df_train = pd.concat(lista_resample)

    df_train['ANHO'] = df_train.index.year
    df_train['MES'] = df_train.index.month
    df_train['HORA'] = df_train.index.hour
    df_train['MINUTO'] = df_train.index.minute
    df_train['DIA'] = df_train.index.day
    df_train['FECHAHORA'] = df_train.index
    df_train.reset_index(inplace = True, drop = True)

    df_train.to_csv('datos/experimento_full/train_resample_'+str(resample_item)+'_minutes.csv')

    # OPTUNA

    for previous_item in previous:
        for horizon_item in horizon:
            
            estacion = 4

            variables = ["ANHO", 'DIA', 'MES', 'HORA', 'MINUTO', 'MP1', 'MP2_5', 'MP10', 'AQI_MP10', 'AQI_MP2_5', 'TIPO', 'TRAFICO', 'HUMEDAD', 
                        'PRESION', 'TEMPERATURA','DIA_SEM']

            dependent = ['AQI_MP2_5']

            number_of_features = len(variables)

            training_days = previous_item
            forecast_days = horizon_item

            if resample_item == 5:
                samples_per_day = 288
            elif resample_item == 30:
                samples_per_day = 48
            elif resample_item == 60:
                samples_per_day = 24

            step = forecast_days*samples_per_day

            train_months = relativedelta(months = 12)

            input_samples = int(samples_per_day * training_days) # cantidad de muestras en 7 dias
            output_samples = int(samples_per_day * forecast_days) # cantidad de muestras en 1 dia
            train_test_samples = int(input_samples + output_samples) # cantidad de datos para el train_test

            X_train, y_train, X_test, y_test = utils_xgboost.get_everything(df_train, 
                                                                estacion,
                                                                train_months, 
                                                                variables, 
                                                                dependent, 
                                                                train_test_samples, 
                                                                input_samples, 
                                                                output_samples, 
                                                                number_of_features,
                                                                step)


            # TESTEO

            trained_xgb_model = joblib.load('models/models_xgboost/experimento_full/xgboost_estacion_4_resample_'+str(resample_item)+'_horizon_'+str(horizon_item)+'_previous_'+str(previous_item)+'.pkl')

            prediction = trained_xgb_model.predict(X_test)
            
            predicciones = {}
            metricas = {}
            # guardamos los valores predecidos vs reales en un diccionario
            
            predicciones[4] = {'real' : y_test, 'prediccion': prediction}


            mean_real = y_test.mean()
            mean_prediction = prediction.mean()

            MAPE = mean_absolute_percentage_error(prediction, y_test)
            MAE = mean_absolute_error(prediction, y_test)
            RMSE = mean_squared_error(prediction, y_test, squared = False)
            
            # guardamos las metricas en un diccionario
            
            metricas[4] = {'MAE': MAE, "MAPE": MAPE, 'RMSE': RMSE, 'Media real' : mean_real, 'Media predecida': mean_prediction}

            df_metricas = pd.DataFrame.from_dict(metricas)

            df_metricas.to_csv('metrics/XGBOOST/experimento_full/testing_xgboost_experimento_full_resample_'+str(resample_item)+'_horizon_'+str(horizon_item)+'_previous_'+str(previous_item)+'.csv')

            list_dfs = []


            d = {'TARGET': predicciones[4]['real'].flatten(), 'FORECAST': predicciones[4]['prediccion'].flatten()}
            df_predicciones = pd.DataFrame(data = d)
            df_predicciones['ESTACION'] = 4

            df_predicciones.to_csv('datos/experimento_full/predicciones_estacion4_resample_'+str(resample_item)+'_horizon_'+str(horizon_item)+'_previous_'+str(previous_item)+'.csv')

            print('testing_'+ str(resample_item)+'_'+str(horizon_item)+'_'+ str(previous_item)+' done!')

# MEJORES PARAMETROS / 10% DE CORRELACION

In [ ]:
horizon = [0.25, 0.5, 1]
resample = [5, 30, 60]
previous = [2, 7, 15]

# Start resampling

for resample_item in resample:

    df_train = datos.copy()

    df_train.drop(['ANHO', 'DIA', 'MES', 'HORA', 'MINUTO', 'DIA_TRAF_COD', 'TRAFICO_COD', 'TIPO_COD',
                'MEDICION_DIA', 'MP1_ANTERIOR', 'MP2_5_ANTERIOR', 'MP10_ANTERIOR',
                'TEMPERATURA_PRONOSTICO', 'HUMEDAD_PRONOSTICO', 'PRESION_PRONOSTICO'], axis = 1, inplace = True)

    df_train = df_train.set_index('FECHAHORA', drop = True)

    lista_resample = []

    if resample_item != 5: 

        r_i = str(resample_item) + 'T'

        for station in range(1,11):
            df_aux = df_train[df_train['ESTACION'] == station]

            df_aux = df_aux.resample(r_i).mean()
            lista_resample.append(df_aux)

        df_train = pd.concat(lista_resample)

    df_train['ANHO'] = df_train.index.year
    df_train['MES'] = df_train.index.month
    df_train['HORA'] = df_train.index.hour
    df_train['MINUTO'] = df_train.index.minute
    df_train['DIA'] = df_train.index.day
    df_train['FECHAHORA'] = df_train.index
    df_train.reset_index(inplace = True, drop = True)

    df_train.to_csv('datos/experimento_full/train_resample_'+str(resample_item)+'_minutes.csv')

    # OPTUNA

    for previous_item in previous:
        for horizon_item in horizon:
            
            estacion = 4

            variables = ['MES', 'MP1', 'MP2_5', 'MP10', 'AQI_MP10', 'AQI_MP2_5', 'HUMEDAD', 
                        'PRESION', 'TEMPERATURA','DIA_SEM']

            dependent = ['AQI_MP2_5']

            number_of_features = len(variables)

            training_days = previous_item
            forecast_days = horizon_item

            if resample_item == 5:
                samples_per_day = 288
            elif resample_item == 30:
                samples_per_day = 48
            elif resample_item == 60:
                samples_per_day = 24

            step = forecast_days*samples_per_day

            train_months = relativedelta(months = 12)

            input_samples = int(samples_per_day * training_days) # cantidad de muestras en 7 dias
            output_samples = int(samples_per_day * forecast_days) # cantidad de muestras en 1 dia
            train_test_samples = int(input_samples + output_samples) # cantidad de datos para el train_test

            X_train, y_train, X_test, y_test = utils_xgboost.get_everything(df_train, 
                                                                estacion,
                                                                train_months, 
                                                                variables, 
                                                                dependent, 
                                                                train_test_samples, 
                                                                input_samples, 
                                                                output_samples, 
                                                                number_of_features,
                                                                step)


            # TESTEO

            trained_xgb_model = joblib.load('models/models_xgboost/experimento_full/xgboost_mejoresparam_estacion_4_resample_'+str(resample_item)+'_horizon_'+str(horizon_item)+'_previous_'+str(previous_item)+'.pkl')

            prediction = trained_xgb_model.predict(X_test)
            
            predicciones = {}
            metricas = {}
            # guardamos los valores predecidos vs reales en un diccionario
            
            predicciones[4] = {'real' : y_test, 'prediccion': prediction}


            mean_real = y_test.mean()
            mean_prediction = prediction.mean()

            MAPE = mean_absolute_percentage_error(prediction, y_test)
            MAE = mean_absolute_error(prediction, y_test)
            RMSE = mean_squared_error(prediction, y_test, squared = False)
            
            # guardamos las metricas en un diccionario
            
            metricas[4] = {'MAE': MAE, "MAPE": MAPE, 'RMSE': RMSE, 'Media real' : mean_real, 'Media predecida': mean_prediction}

            df_metricas = pd.DataFrame.from_dict(metricas)

            df_metricas.to_csv('metrics/XGBOOST/experimento_full/testing_xgboost_MEJORESPARAM10_experimento_full_resample_'+str(resample_item)+'_horizon_'+str(horizon_item)+'_previous_'+str(previous_item)+'.csv')

            list_dfs = []


            d = {'TARGET': predicciones[4]['real'].flatten(), 'FORECAST': predicciones[4]['prediccion'].flatten()}
            df_predicciones = pd.DataFrame(data = d)
            df_predicciones['ESTACION'] = 4

            df_predicciones.to_csv('datos/experimento_full/predicciones_estacion4_MEJORESPARAM10_resample_'+str(resample_item)+'_horizon_'+str(horizon_item)+'_previous_'+str(previous_item)+'.csv')

            print('testing_'+ str(resample_item)+'_'+str(horizon_item)+'_'+ str(previous_item)+' done!')

# MEJORES PARAMETROS - 20% DE CORRELACION

In [ ]:
horizon = [0.25, 0.5, 1]
resample = [5, 30, 60]
previous = [2, 7, 15]

# Start resampling

for resample_item in resample:

    df_train = datos.copy()

    df_train.drop(['ANHO', 'DIA', 'MES', 'HORA', 'MINUTO', 'DIA_TRAF_COD', 'TRAFICO_COD', 'TIPO_COD',
                'MEDICION_DIA', 'MP1_ANTERIOR', 'MP2_5_ANTERIOR', 'MP10_ANTERIOR',
                'TEMPERATURA_PRONOSTICO', 'HUMEDAD_PRONOSTICO', 'PRESION_PRONOSTICO'], axis = 1, inplace = True)

    df_train = df_train.set_index('FECHAHORA', drop = True)

    lista_resample = []

    if resample_item != 5: 

        r_i = str(resample_item) + 'T'

        for station in range(1,11):
            df_aux = df_train[df_train['ESTACION'] == station]

            df_aux = df_aux.resample(r_i).mean()
            lista_resample.append(df_aux)

        df_train = pd.concat(lista_resample)

    df_train['ANHO'] = df_train.index.year
    df_train['MES'] = df_train.index.month
    df_train['HORA'] = df_train.index.hour
    df_train['MINUTO'] = df_train.index.minute
    df_train['DIA'] = df_train.index.day
    df_train['FECHAHORA'] = df_train.index
    df_train.reset_index(inplace = True, drop = True)

    df_train.to_csv('datos/experimento_full/train_resample_'+str(resample_item)+'_minutes.csv')

    # OPTUNA

    for previous_item in previous:
        for horizon_item in horizon:
            
            estacion = 4

            variables = ['MP1', 'MP2_5', 'MP10', 'AQI_MP10', 'AQI_MP2_5', 
                        'PRESION', 'DIA_SEM']

            dependent = ['AQI_MP2_5']

            number_of_features = len(variables)

            training_days = previous_item
            forecast_days = horizon_item

            if resample_item == 5:
                samples_per_day = 288
            elif resample_item == 30:
                samples_per_day = 48
            elif resample_item == 60:
                samples_per_day = 24

            step = forecast_days*samples_per_day

            train_months = relativedelta(months = 12)

            input_samples = int(samples_per_day * training_days) # cantidad de muestras en 7 dias
            output_samples = int(samples_per_day * forecast_days) # cantidad de muestras en 1 dia
            train_test_samples = int(input_samples + output_samples) # cantidad de datos para el train_test

            X_train, y_train, X_test, y_test = utils_xgboost.get_everything(df_train, 
                                                                estacion,
                                                                train_months, 
                                                                variables, 
                                                                dependent, 
                                                                train_test_samples, 
                                                                input_samples, 
                                                                output_samples, 
                                                                number_of_features,
                                                                step)


            def objective(trial):
    
                params = {
                    'max_depth': trial.suggest_int('max_depth', 1, 9),
                    'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
                    'n_estimators': trial.suggest_int('n_estimators', 10, 300),
                    'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
                    'gamma': trial.suggest_loguniform('gamma', 1e-5, 1.0),
                    'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
                    'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
                }
                
                optuna_xgb_model = xgb.XGBRegressor(** params)
                
                trained_xgb_model = MultiOutputRegressor(optuna_xgb_model).fit(X_train , y_train)

                prediction = trained_xgb_model.predict(X_test)

                MAPE = mean_absolute_percentage_error(prediction, y_test)
                print('MAPE: ', MAPE)
                MAE = np.mean(np.abs(prediction - y_test))
                print('MAE: ', MAE)

                MSE = mean_squared_error(prediction, y_test)
                
                return MSE

            study = optuna.create_study(direction='minimize')

            study.optimize(objective, n_trials = 100 )

            trial = study.best_trial

            joblib.dump(study, "optuna_studies/XGBOOST/experimento_full/study_XGBOOST_mejoresparam20_resample_"+str(resample_item)+"_horizon_"+str(horizon_item)+"_previous_"+str(previous_item)+".pkl")

            # VALIDATION

            params = study.best_params

            # study = joblib.load("optuna_studies/XGBOOST/experimento_full/study_XGBOOST_resample_"+str(resample_item)+"_horizon_"+str(horizon_item)+"_previous_"+str(previous_item)+".pkl")

            xgb_model = xgb.XGBRegressor(** params)

            trained_xgb_model = MultiOutputRegressor(xgb_model).fit(X_train , y_train)
            
            prediction = trained_xgb_model.predict(X_test)
            
            predicciones = {}
            metricas = {}
            # guardamos los valores predecidos vs reales en un diccionario
            
            predicciones[4] = {'real' : y_test, 'prediccion': prediction}


            mean_real = y_test.mean()
            mean_prediction = prediction.mean()

            MAPE = mean_absolute_percentage_error(prediction, y_test)
            MAE =100 * mean_absolute_error(prediction, y_test)
            RMSE = mean_squared_error(prediction, y_test, squared = False)
            
            # guardamos las metricas en un diccionario
            
            metricas[4] = {'MAE': MAE, "MAPE": MAPE, 'RMSE': RMSE, 'Media real' : mean_real, 'Media predecida': mean_prediction}

            df_metricas = pd.DataFrame.from_dict(metricas)

            df_metricas.to_csv('metrics/XGBOOST/experimento_full/testing_xgboost_MEJORESPARAM20_experimento_full_resample_'+str(resample_item)+'_horizon_'+str(horizon_item)+'_previous_'+str(previous_item)+'.csv')

            list_dfs = []


            d = {'TARGET': predicciones[4]['real'].flatten(), 'FORECAST': predicciones[4]['prediccion'].flatten()}
            df_predicciones = pd.DataFrame(data = d)
            df_predicciones['ESTACION'] = 4

            df_predicciones.to_csv('datos/experimento_full/predicciones_estacion4_MEJORESPARAM20_resample_'+str(resample_item)+'_horizon_'+str(horizon_item)+'_previous_'+str(previous_item)+'.csv')


# MEJORES PARAMETROS - 40% DE CORRELACION

In [ ]:
horizon = [0.25, 0.5, 1]
resample = [5, 30, 60]
previous = [2, 7, 15]

# Start resampling

for resample_item in resample:

    df_train = datos.copy()

    df_train.drop(['ANHO', 'DIA', 'MES', 'HORA', 'MINUTO', 'DIA_TRAF_COD', 'TRAFICO_COD', 'TIPO_COD',
                'MEDICION_DIA', 'MP1_ANTERIOR', 'MP2_5_ANTERIOR', 'MP10_ANTERIOR',
                'TEMPERATURA_PRONOSTICO', 'HUMEDAD_PRONOSTICO', 'PRESION_PRONOSTICO'], axis = 1, inplace = True)

    df_train = df_train.set_index('FECHAHORA', drop = True)

    lista_resample = []

    if resample_item != 5: 

        r_i = str(resample_item) + 'T'

        for station in range(1,11):
            df_aux = df_train[df_train['ESTACION'] == station]

            df_aux = df_aux.resample(r_i).mean()
            lista_resample.append(df_aux)

        df_train = pd.concat(lista_resample)

    df_train['ANHO'] = df_train.index.year
    df_train['MES'] = df_train.index.month
    df_train['HORA'] = df_train.index.hour
    df_train['MINUTO'] = df_train.index.minute
    df_train['DIA'] = df_train.index.day
    df_train['FECHAHORA'] = df_train.index
    df_train.reset_index(inplace = True, drop = True)

    #df_train.to_csv('datos/experimento_full/train_resample_'+str(resample_item)+'_minutes.csv')

    # OPTUNA

    for previous_item in previous:
        for horizon_item in horizon:
            
            estacion = 4

            variables = ['MP1', 'MP2_5', 'AQI_MP10', 'AQI_MP2_5']

            dependent = ['AQI_MP2_5']

            number_of_features = len(variables)

            training_days = previous_item
            forecast_days = horizon_item

            if resample_item == 5:
                samples_per_day = 288
            elif resample_item == 30:
                samples_per_day = 48
            elif resample_item == 60:
                samples_per_day = 24

            step = forecast_days*samples_per_day

            train_months = relativedelta(months = 12)

            input_samples = int(samples_per_day * training_days) # cantidad de muestras en 7 dias
            output_samples = int(samples_per_day * forecast_days) # cantidad de muestras en 1 dia
            train_test_samples = int(input_samples + output_samples) # cantidad de datos para el train_test

            X_train, y_train, X_test, y_test = utils_xgboost.get_everything(df_train, 
                                                                estacion,
                                                                train_months, 
                                                                variables, 
                                                                dependent, 
                                                                train_test_samples, 
                                                                input_samples, 
                                                                output_samples, 
                                                                number_of_features,
                                                                step)


            def objective(trial):
    
                params = {
                    'max_depth': trial.suggest_int('max_depth', 1, 9),
                    'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
                    'n_estimators': trial.suggest_int('n_estimators', 10, 300),
                    'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
                    'gamma': trial.suggest_loguniform('gamma', 1e-5, 1.0),
                    'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
                    'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
                }
                
                optuna_xgb_model = xgb.XGBRegressor(** params)
                
                trained_xgb_model = MultiOutputRegressor(optuna_xgb_model).fit(X_train , y_train)

                prediction = trained_xgb_model.predict(X_test)

                MAPE = mean_absolute_percentage_error(prediction, y_test)
                print('MAPE: ', MAPE)
                MAE = np.mean(np.abs(prediction - y_test))
                print('MAE: ', MAE)

                MSE = mean_squared_error(prediction, y_test)
                
                return MSE

            study = optuna.create_study(direction='minimize')

            study.optimize(objective, n_trials = 100 )

            trial = study.best_trial

            joblib.dump(study, "optuna_studies/XGBOOST/experimento_full/NORMALIZADO_study_XGBOOST_mejoresparam40_resample_"+str(resample_item)+"_horizon_"+str(horizon_item)+"_previous_"+str(previous_item)+".pkl")

            # VALIDATION

            params = study.best_params

            # study = joblib.load("optuna_studies/XGBOOST/experimento_full/study_XGBOOST_resample_"+str(resample_item)+"_horizon_"+str(horizon_item)+"_previous_"+str(previous_item)+".pkl")

            xgb_model = xgb.XGBRegressor(** params)

            trained_xgb_model = MultiOutputRegressor(xgb_model).fit(X_train , y_train)
            
            prediction = trained_xgb_model.predict(X_test)
            
            predicciones = {}
            metricas = {}
            # guardamos los valores predecidos vs reales en un diccionario
            
            predicciones[4] = {'real' : y_test, 'prediccion': prediction}


            mean_real = y_test.mean()
            mean_prediction = prediction.mean()

            MAPE = mean_absolute_percentage_error(prediction, y_test) * 100
            MAE = mean_absolute_error(prediction, y_test)
            RMSE = mean_squared_error(prediction, y_test, squared = False)
            
            # guardamos las metricas en un diccionario
            
            metricas[4] = {'MAE': MAE, "MAPE": MAPE, 'RMSE': RMSE, 'Media real' : mean_real, 'Media predecida': mean_prediction}

            df_metricas = pd.DataFrame.from_dict(metricas)

            df_metricas.to_csv('metrics/XGBOOST/experimento_full/testing_xgboost_MEJORESPARAM_40_experimento_full_resample_'+str(resample_item)+'_horizon_'+str(horizon_item)+'_previous_'+str(previous_item)+'.csv')

            list_dfs = []


            d = {'TARGET': predicciones[4]['real'].flatten(), 'FORECAST': predicciones[4]['prediccion'].flatten()}
            df_predicciones = pd.DataFrame(data = d)
            df_predicciones['ESTACION'] = 4

            df_predicciones.to_csv('datos/experimento_full/predicciones_estacion4_MEJORESPARAM_40_resample_'+str(resample_item)+'_horizon_'+str(horizon_item)+'_previous_'+str(previous_item)+'.csv')

# Probamos reentrenar el mejor modelo para 6hs normalizando

In [3]:
horizon = [0.25]
resample = [5]
previous = [2]

# Start resampling

for resample_item in resample:

    df_train = datos.copy()

    df_train.drop(['ANHO', 'DIA', 'MES', 'HORA', 'MINUTO', 'DIA_TRAF_COD', 'TRAFICO_COD', 'TIPO_COD',
                'MEDICION_DIA', 'MP1_ANTERIOR', 'MP2_5_ANTERIOR', 'MP10_ANTERIOR',
                'TEMPERATURA_PRONOSTICO', 'HUMEDAD_PRONOSTICO', 'PRESION_PRONOSTICO'], axis = 1, inplace = True)

    df_train = df_train.set_index('FECHAHORA', drop = True)

    lista_resample = []

    if resample_item != 5: 

        r_i = str(resample_item) + 'T'

        for station in range(1,11):
            df_aux = df_train[df_train['ESTACION'] == station]

            df_aux = df_aux.resample(r_i).mean()
            lista_resample.append(df_aux)

        df_train = pd.concat(lista_resample)

    df_train['ANHO'] = df_train.index.year
    df_train['MES'] = df_train.index.month
    df_train['HORA'] = df_train.index.hour
    df_train['MINUTO'] = df_train.index.minute
    df_train['DIA'] = df_train.index.day
    df_train['FECHAHORA'] = df_train.index
    df_train.reset_index(inplace = True, drop = True)

    #df_train.to_csv('datos/experimento_full/train_resample_'+str(resample_item)+'_minutes.csv')

    # OPTUNA

    for previous_item in previous:
        for horizon_item in horizon:
            
            estacion = 4

            variables = ['MP1', 'MP2_5', 'AQI_MP10', 'AQI_MP2_5']

            dependent = ['AQI_MP2_5']

            number_of_features = len(variables)

            training_days = previous_item
            forecast_days = horizon_item

            if resample_item == 5:
                samples_per_day = 288
            elif resample_item == 30:
                samples_per_day = 48
            elif resample_item == 60:
                samples_per_day = 24

            step = forecast_days*samples_per_day

            train_months = relativedelta(months = 12)

            input_samples = int(samples_per_day * training_days) # cantidad de muestras en 7 dias
            output_samples = int(samples_per_day * forecast_days) # cantidad de muestras en 1 dia
            train_test_samples = int(input_samples + output_samples) # cantidad de datos para el train_test

            X_train, y_train, X_test, y_test = utils_xgboost.get_everything(df_train, 
                                                                estacion,
                                                                train_months, 
                                                                variables, 
                                                                dependent, 
                                                                train_test_samples, 
                                                                input_samples, 
                                                                output_samples, 
                                                                number_of_features,
                                                                step)


            def objective(trial):
    
                params = {
                    'max_depth': trial.suggest_int('max_depth', 1, 9),
                    'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
                    'n_estimators': trial.suggest_int('n_estimators', 10, 300),
                    'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
                    'gamma': trial.suggest_loguniform('gamma', 1e-5, 1.0),
                    'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
                    'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
                }
                
                optuna_xgb_model = xgb.XGBRegressor(** params)
                
                trained_xgb_model = MultiOutputRegressor(optuna_xgb_model).fit(X_train , y_train)

                prediction = trained_xgb_model.predict(X_test)

                MAPE = mean_absolute_percentage_error(prediction, y_test)
                print('MAPE: ', MAPE)
                MAE = np.mean(np.abs(prediction - y_test))
                print('MAE: ', MAE)

                MSE = mean_squared_error(prediction, y_test)
                
                return MSE

            study = optuna.create_study(direction='minimize')

            study.optimize(objective, n_trials = 100 )

            trial = study.best_trial

            joblib.dump(study, "optuna_studies/XGBOOST/experimento_full/NORMALIZADO_study_XGBOOST_mejoresparam40_resample_"+str(resample_item)+"_horizon_"+str(horizon_item)+"_previous_"+str(previous_item)+".pkl")

            # VALIDATION

            params = study.best_params

            # study = joblib.load("optuna_studies/XGBOOST/experimento_full/study_XGBOOST_resample_"+str(resample_item)+"_horizon_"+str(horizon_item)+"_previous_"+str(previous_item)+".pkl")

            xgb_model = xgb.XGBRegressor(** params)

            trained_xgb_model = MultiOutputRegressor(xgb_model).fit(X_train , y_train)
            
            prediction = trained_xgb_model.predict(X_test)
            
            predicciones = {}
            metricas = {}
            # guardamos los valores predecidos vs reales en un diccionario
            
            predicciones[4] = {'real' : y_test, 'prediccion': prediction}


            mean_real = y_test.mean()
            mean_prediction = prediction.mean()

            MAPE = mean_absolute_percentage_error(prediction, y_test) * 100
            MAE = mean_absolute_error(prediction, y_test)
            RMSE = mean_squared_error(prediction, y_test, squared = False)
            
            # guardamos las metricas en un diccionario
            
            metricas[4] = {'MAE': MAE, "MAPE": MAPE, 'RMSE': RMSE, 'Media real' : mean_real, 'Media predecida': mean_prediction}

            df_metricas = pd.DataFrame.from_dict(metricas)

            df_metricas.to_csv('metrics/XGBOOST/experimento_full/NORMALIZADO_testing_xgboost_MEJORESPARAM_40_experimento_full_resample_'+str(resample_item)+'_horizon_'+str(horizon_item)+'_previous_'+str(previous_item)+'.csv')

            list_dfs = []


            d = {'TARGET': predicciones[4]['real'].flatten(), 'FORECAST': predicciones[4]['prediccion'].flatten()}
            df_predicciones = pd.DataFrame(data = d)
            df_predicciones['ESTACION'] = 4

            df_predicciones.to_csv('datos/experimento_full/NORMALIZADO_predicciones_estacion4_MEJORESPARAM_40_resample_'+str(resample_item)+'_horizon_'+str(horizon_item)+'_previous_'+str(previous_item)+'.csv')

[I 2023-04-08 15:57:06,008] A new study created in memory with name: no-name-1b53e661-67df-48c2-829f-2e7af4338d9e
[I 2023-04-08 15:57:11,061] Trial 0 finished with value: 262.963196866539 and parameters: {'max_depth': 5, 'learning_rate': 0.8270984379558769, 'n_estimators': 29, 'min_child_weight': 10, 'gamma': 0.550364140892195, 'subsample': 0.03012258486675799, 'colsample_bytree': 0.6799456983913087}. Best is trial 0 with value: 262.963196866539.


MAPE:  1.271467002785484
MAE:  12.688769949954724


[I 2023-04-08 15:57:19,310] Trial 1 finished with value: 115.76887653199866 and parameters: {'max_depth': 5, 'learning_rate': 0.38318500218492396, 'n_estimators': 62, 'min_child_weight': 5, 'gamma': 2.3345896328274177e-05, 'subsample': 0.01769370618583572, 'colsample_bytree': 0.38763273017704225}. Best is trial 1 with value: 115.76887653199866.


MAPE:  0.7068334443712886
MAE:  8.383610496162024


[I 2023-04-08 15:57:36,405] Trial 2 finished with value: 42.2483687874146 and parameters: {'max_depth': 1, 'learning_rate': 0.012307010411163694, 'n_estimators': 196, 'min_child_weight': 5, 'gamma': 0.00017756145201405863, 'subsample': 0.08824127587911312, 'colsample_bytree': 0.2649579880383336}. Best is trial 2 with value: 42.2483687874146.


...

MAPE:  0.22630969173495283
MAE:  4.534630082315124


[I 2023-04-08 16:46:21,191] Trial 98 finished with value: 17.759006748635944 and parameters: {'max_depth': 7, 'learning_rate': 0.05666988381639304, 'n_estimators': 48, 'min_child_weight': 6, 'gamma': 1.9272197436713403e-05, 'subsample': 0.286934025797607, 'colsample_bytree': 0.14649078617142328}. Best is trial 92 with value: 16.98144161507129.


MAPE:  0.12526095752894134
MAE:  2.7703845053725176


[I 2023-04-08 16:46:31,182] Trial 99 finished with value: 23.884540754666734 and parameters: {'max_depth': 3, 'learning_rate': 0.06258599190808524, 'n_estimators': 69, 'min_child_weight': 7, 'gamma': 4.715310538548398e-05, 'subsample': 0.4510411782082597, 'colsample_bytree': 0.10256746599773371}. Best is trial 92 with value: 16.98144161507129.


MAPE:  0.14501902188658777
MAE:  3.41334038969134
